In [12]:
!pip install transformers datasets torch

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("imdb")

def preprocess_function(examples):
    inputs = ["Review: " + review + "\nSentiment:" for review in examples["text"]]
    tokenized_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["text", "label"])

train_dataset = tokenized_datasets["train"].select(range(5000))
eval_dataset = tokenized_datasets["test"].select(range(5000))


training_args = TrainingArguments(
    output_dir="./results_imdb",
    eval_strategy="epoch",
    per_device_train_batch_size=2,
    num_train_epochs=1,
)


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_imdb_model")

def generate_sentiment(review):
    prompt = "Review: " + review + "\nSentiment:"
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(model.device)
    output = model.generate(input_ids, max_length=50)
    return tokenizer.decode(output[0], skip_special_tokens=True)

review_example = "This movie was a complete waste of time."
print("Review:", review_example)
print("Generated output:", generate_sentiment(review_example))


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,3.608000,3.553339


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Review: This movie was a complete waste of time.
Generated output: Review: This movie was a complete waste of time.
Sentiment: I'm a big fan of the original "The Matrix" and I'm not a big fan of the sequels. I'm a big fan of the original "The Matrix"


In [ ]:
from transformers import pipeline



explainer = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha")



prompt = "Review: This movie was a complete waste of time. Whats the sentiment of this review"

explanation = explainer(prompt, max_length=100)

print(explanation[0]["generated_text"])

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]